In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
from datatable import dt, f, by, g, join, sort, update, ifelse

In [ ]:
homer_path = ''
# homer_path = 'export PATH=$PATH:/sc/projects/sci-kretzmer/chair/software/tools/homer/bin/;'
env = ''
# env = 'eval "$(conda shell.bash hook)";conda activate /sc/projects/sci-kretzmer/chair/software/conda/metilene3;export MPLBACKEND=qtagg;'

In [ ]:
os.system(env+'python ../metilene3/metilene3.py \
    -i ./data/GSE222147_pdac.input.tsv \
    -o ./PDAC \
    -t 16 \
    -n 4 \
    -plot True -anno hg19\
')

In [ ]:
dmrs = pd.read_table('./PDAC/DMRs.tsv').sort_values(['chr','start','stop'])
dmrs['strand'] = '+'
dmrs['stop_1'] = dmrs['stop']+1

path = './PDAC/motif/'
os.system('mkdir '+path)
for i in ['P2|3|3|3']:
    dmrs.loc[(dmrs['meandiffabs']>=0.5)&(dmrs['DMTree'].fillna('').str.contains(i.replace('|','\|')+','))][['chr','start','stop_1','strand']].\
        to_csv(path+i.replace('|','_')+'.bed', sep='\t', header=False)
    dmrs.loc[(dmrs['meandiffabs']>=0.5)&(~dmrs['DMTree'].fillna('').str.contains(i.replace('|','\|')+','))][['chr','start','stop_1','strand']].\
        to_csv(path+i.replace('|','_')+'.anti.bed', sep='\t', header=False)
    os.system(env+homer_path+'findMotifsGenome.pl '+path+i.replace('|','_')+'.bed'+\
                  ' hg19'+\
                  ' '+path+i.replace('|','_')+' -bits -size 250  -bg '+path+i.replace('|','_')+'.anti.bed ')

In [ ]:
# PDAC-hypo DMRs
dmrs.loc[(dmrs['meandiffabs']>=0.5)&(dmrs['DMTree'].fillna('').str.contains("P2|3|3|3".replace('|','\|')+','))]['sig.comparison'].value_counts()

In [ ]:
motif_id = {}

os.system('mkdir ./PDAC/motif/find')

for motif in os.listdir('./PDAC/motif/P2_3_3_3/knownResults'):
    if motif.split('.')[-1]=='motif':
        motif_id[motif] = pd.read_table('./PDAC/motif/P2_3_3_3/knownResults/'+motif,nrows=0).columns[1]

        os.system(env+homer_path+'findMotifsGenome.pl ./PDAC/motif/P2_3_3_3.bed'+\
                  ' hg19'+\
                  ' ./PDAC/motif/find/'+motif+' \
                  -bits -size 250 \
                  -find  ./PDAC/motif/P2_3_3_3/knownResults/'+motif+' > \
                  ./PDAC/motif/find/'+motif+'.find.PDAC.tsv')
        
        os.system(env+homer_path+'findMotifsGenome.pl ./PDAC/motif/P2_3_3_3.anti.bed'+\
                  ' hg19'+\
                  ' ./PDAC/motif/find/'+motif+'-anti \
                  -bits -size 250 \
                  -find  ./PDAC/motif/P2_3_3_3/knownResults/'+motif+' > \
                  ./PDAC/motif/find/'+motif+'.find.PDAC.anti.tsv')
        
        print(motif,motif_id[motif])

In [ ]:
pdac_motifs = pd.read_table('./PDAC/motif/P2_3_3_3.bed', header=None)

for i in motif_id.keys():
    tmp = pd.read_table('./PDAC/motif/find/'+i+'.find.PDAC.tsv', index_col='PositionID')
    pdac_motifs[motif_id[i]] = pdac_motifs[0].map(tmp['Offset'].to_dict())

pdac_motifs

In [ ]:
pdac_anti_motifs = pd.read_table('./PDAC/motif/P2_3_3_3.anti.bed', header=None)

for i in motif_id.keys():
    tmp = pd.read_table('./PDAC/motif/find/'+i+'.find.PDAC.anti.tsv', index_col='PositionID')
    pdac_anti_motifs[motif_id[i]] = pdac_anti_motifs[0].map(tmp['Offset'].to_dict())

pdac_anti_motifs

In [ ]:
cls = sns.clustermap(pdac_motifs[motif_id.values()].isna().corr(),\
               vmin = -0.3, vmax = 0.3, center = 0, cmap='coolwarm', figsize=[20,20])
sns.clustermap(pdac_anti_motifs[motif_id.values()].isna().corr(),\
               vmin = -0.3, vmax = 0.3, center = 0, cmap='coolwarm', figsize=[20,20],\
               col_linkage=cls.dendrogram_col.linkage, row_linkage=cls.dendrogram_row.linkage
              )
sns.clustermap(pdac_motifs[motif_id.values()].isna().corr()-pdac_anti_motifs[motif_id.values()].isna().corr(),\
               vmin = -0.3, vmax = 0.3, center = 0, cmap='coolwarm', figsize=[20,20], method='ward',\
               col_linkage=cls.dendrogram_col.linkage, row_linkage=cls.dendrogram_row.linkage
              )

In [ ]:
dmrs = pd.read_table('./PDAC/DMRs.tsv')
dmrs['strand'] = '+'
dmrs['stop'] = dmrs['stop']+1
dmrs.loc[dmrs['meandiffabs']>=0.5][['chr','start','stop','strand']].to_csv('./PDAC/motif/DMRs.high.bed', sep='\t', header=False)
dmrs.loc[dmrs['meandiffabs']>=0.5][['chr','start','stop','strand']]

In [ ]:
motifs_toshow = {
    'NFATC1':'./PDAC/motif/P2_3_3_3/knownResults/known6.motif',\
    'NFKB2':'./PDAC/motif/P2_3_3_3/knownResults/known3.motif'\
}

In [ ]:
for i in motifs_toshow.keys():  
    print(i, motif_id[motifs_toshow[i].split('/')[-1]])
    os.system(env+homer_path+'findMotifsGenome.pl ./PDAC/motif/DMRs.high.bed'+\
                  ' hg19'+\
                  ' ./PDAC/motif/'+i+' \
                  -bits -size 250 \
                  -find '+motifs_toshow[i]+' > \
                  ./PDAC/motif/'+i+'.find.allhighDMRs.tsv')

In [ ]:
allhighdmrs_motifs = {}

for i in motifs_toshow.keys():
    tmp = pd.read_table('./PDAC/motif/'+i+'.find.allhighDMRs.tsv', index_col='PositionID')
    allhighdmrs_motifs[i+'-dmrs'] = dmrs.loc[dmrs.index.isin(set(tmp.index.astype(int)))]
    allhighdmrs_motifs[i+'-dmrs']['DMRid'] = allhighdmrs_motifs[i+'-dmrs'].index
    allhighdmrs_motifs[i+'-dmrs'][5] = allhighdmrs_motifs[i+'-dmrs']['DMRid'].map(tmp['Offset'].to_dict()).astype(int)
    allhighdmrs_motifs[i+'-dmrs'][6] = ((allhighdmrs_motifs[i+'-dmrs']['start']+allhighdmrs_motifs[i+'-dmrs']['stop'])/2).astype(int)+allhighdmrs_motifs[i+'-dmrs'][5]
    allhighdmrs_motifs[i+'-dmrs'][7] = allhighdmrs_motifs[i+'-dmrs'][6]+1
    allhighdmrs_motifs[i+'-dmrs'][['chr',6,7]].to_csv('./PDAC/motif/'+i+'.find.allhighDMRs.emap.tsv',
                                           sep='\t', index=False,header=False)

In [ ]:
tmp = pd.merge(allhighdmrs_motifs['NFATC1-dmrs'],allhighdmrs_motifs['NFKB2-dmrs'],on=['chr','start','stop'])[['chr','6_x','7_x','6_y','7_y']]
tmp[6] = tmp[['6_x','7_x','6_y','7_y']].T.min()
tmp[7] = tmp[['6_x','7_x','6_y','7_y']].T.max()
tmp.to_csv('./PDAC/motif/NFKB2_NFATC1.find.allhighDMRs.emap.tsv',
                                           sep='\t', index=False,header=False)

In [ ]:
highdmrs = dmrs.loc[(dmrs['meandiffabs']>=0.5)]
for i in ['NFATC1-dmrs','NFKB2-dmrs']:
    highdmrs[i] = (
        highdmrs['chr']+':'+highdmrs['start'].astype(str)
    ).isin(
        set(allhighdmrs_motifs[i]['chr']+':'+allhighdmrs_motifs[i]['start'].astype(str))
    )

print(st.fisher_exact(
    pd.crosstab(highdmrs['DMTree'].fillna('').str.contains('P2\|3\|3\|3'),\
    highdmrs['NFATC1-dmrs']&highdmrs['NFKB2-dmrs'])
))
pd.crosstab(highdmrs['DMTree'].fillna('').str.contains('P2\|3\|3\|3'),\
highdmrs['NFATC1-dmrs']&highdmrs['NFKB2-dmrs'])

In [ ]:
i = 'NFATC1-dmrs'
print(st.fisher_exact(
    pd.crosstab(highdmrs['DMTree'].fillna('').str.contains('P2\|3\|3\|3'),\
    highdmrs[i])
))
pd.crosstab(highdmrs['DMTree'].fillna('').str.contains('P2\|3\|3\|3'),\
highdmrs[i]).sort_index(ascending=False).T.sort_index(ascending=False).T

In [ ]:
i = 'NFKB2-dmrs'
print(st.fisher_exact(
    pd.crosstab(highdmrs['DMTree'].fillna('').str.contains('P2\|3\|3\|3'),\
    highdmrs[i])
))
pd.crosstab(highdmrs['DMTree'].fillna('').str.contains('P2\|3\|3\|3'),\
highdmrs[i]).sort_index(ascending=False).T.sort_index(ascending=False).T

In [ ]:
gepia2 = pd.read_table('./gepia2_paad_degs.txt', index_col=0, comment='#')
gepia2.columns = ['transcript','PAAD','Normal','Log2FC','P-value']
gepia2['DEcat'] = -1*(gepia2['Log2FC']<-1)+(gepia2['Log2FC']>1)
gepia2['SYMBOL'] = gepia2.index
gepia2

In [ ]:
venn2tab = highdmrs[['chr','start','stop','meandiffabs','SYMBOL','Hypo-groups','Int-groups','Hyper-groups','NFATC1-dmrs','NFKB2-dmrs']].sort_values(['chr','start'])
for i in ['Hypo-groups','Int-groups','Hyper-groups']:
    venn2tab[i] = venn2tab[i].str.replace('G0','PDAC')
    venn2tab[i] = venn2tab[i].str.replace('G1','PanIN')
    venn2tab[i] = venn2tab[i].str.replace('G2','Acinar')
    venn2tab[i] = venn2tab[i].str.replace('G3','Duct')
venn2tab

In [ ]:
volcano2tab = pd.merge(gepia2,venn2tab,on='SYMBOL',how='left')[['transcript','Log2FC','P-value','SYMBOL',\
                                       'chr','start','stop','meandiffabs','Hypo-groups','Int-groups','Hyper-groups','NFATC1-dmrs','NFKB2-dmrs']].sort_values('P-value')
volcano2tab['P-value'] = -volcano2tab['P-value'].apply(np.log10)
volcano2tab = volcano2tab.loc[volcano2tab['NFATC1-dmrs']|volcano2tab['NFKB2-dmrs']].sort_values(['NFATC1-dmrs','NFKB2-dmrs'])
volcano2tab

In [ ]:
print(st.fisher_exact(pd.crosstab(volcano2tab['Log2FC']>0, volcano2tab['Hypo-groups'].str.contains('PDAC'))))
pd.crosstab(volcano2tab['Log2FC']>0, volcano2tab['Hypo-groups'].str.contains('PDAC'))

In [ ]:
plt.subplots(figsize=[3,3])
sns.scatterplot(x=gepia2['Log2FC'],y=-gepia2['P-value'].apply(np.log10), s=1,color='lightgrey')

volcano2tab['shape2'] = volcano2tab['NFATC1-dmrs']*1+volcano2tab['NFKB2-dmrs']*2

sns.scatterplot(x=volcano2tab['Log2FC'],\
                y=volcano2tab['P-value'], 
                s=10,color='grey',style=volcano2tab['shape2'],markers={1:'X',2:'P',3:'s'},legend=None)
sns.scatterplot(x=volcano2tab.loc[volcano2tab['Log2FC']>1]['Log2FC'],\
                y=volcano2tab.loc[volcano2tab['Log2FC']>1]['P-value'], 
                s=10,color='red',style=volcano2tab.loc[volcano2tab['Log2FC']>1]['shape2'],markers={1:'X',2:'P',3:'s'},legend=None)
sns.scatterplot(x=volcano2tab.loc[volcano2tab['Log2FC']<-1]['Log2FC'],\
                y=volcano2tab.loc[volcano2tab['Log2FC']<-1]['P-value'], 
                s=10,color='blue',style=volcano2tab.loc[volcano2tab['Log2FC']<-1]['shape2'],markers={1:'X',2:'P',3:'s'},legend=None)
plt.xlim([-12,12])
plt.yticks([0,50,100])

In [ ]:
met = dt.fread('./data/GSE222147_pdac.input.tsv')
metcols = ['chr','pos']
cpgpos = met[:, [metcols[0], metcols[1]]].to_pandas()
cpgpos['start'] = cpgpos[metcols[1]]
cpgpos['end'] = cpgpos[metcols[1]]+1
cpgpos

In [ ]:
met_pandas = met.to_pandas()
met_pandas

In [ ]:
for i in ['PDAC','PanIN','Acinar','Duct']:
    met_pandas['pos0'] = met_pandas['pos']-1
    met_pandas['mean_'+i] = met_pandas[met_pandas.columns[met_pandas.columns.str.contains('='+i)]].T.mean()

met_pandas[['chr','pos','mean_PDAC','mean_PanIN','mean_Acinar','mean_Duct']].to_csv(\
    './PDAC/motif/groupmean.met.csv')

In [ ]:
os.system('Rscript ./enrichedheatmap.r')